In [2]:
import os
import json

res_dir = '/root/minidroid/result/sensi_apis'
all_res = {}
for file in os.listdir(res_dir):
    with open(os.path.join(res_dir, file), 'r') as fp:
        data = json.load(fp)
    all_res[file.replace('.json', '')] = data

In [4]:
import pprint
sensi_apis_count = {}
reachable_sensi_apis_count = {}
unreachable_sensi_apis_count = {}

for res in all_res.values():
    for page in res.keys():
        apis = res[page]
        for api in apis.keys():
            if api not in sensi_apis_count.keys():
                sensi_apis_count[api] = 1
            else:
                sensi_apis_count[api] += 1
            if apis[api] is None:
                if api not in unreachable_sensi_apis_count.keys():
                    unreachable_sensi_apis_count[api] = 1
                else:
                    unreachable_sensi_apis_count[api] += 1
            else:
                if api not in reachable_sensi_apis_count.keys():
                    reachable_sensi_apis_count[api] = 1
                else:
                    reachable_sensi_apis_count[api] += 1

# pprint.pprint(sensi_apis_count)
# pprint.pprint(reachable_sensi_apis_count)
# pprint.pprint(unreachable_sensi_apis_count)

counts = 0
for count in unreachable_sensi_apis_count.values():
    counts += count
print(counts)


13172
